# AWS Rekognition PPE
## Jeremy Montgomery, Se Eun Kim, Isaac Cha

### Step 1. Import and Initialize necessary resources.

We will use `boto3` library to use AWS services.

In [1]:
import boto3

We first initialize AWS Rekognition service using `boto3`.

In [2]:
client=boto3.client('rekognition')

Then we access our S3 bucket named `qtm350-fp` where our image data is stored.

In [4]:
s3_resource = boto3.resource('s3')
my_bucket = s3_resource.Bucket('qtm350-fp')
summaries = my_bucket.objects.all()
summaries

s3.Bucket.objectsCollection(s3.Bucket(name='qtm350-fp'), s3.ObjectSummary)

We then extract the names of the image in our S3.

In [28]:
images = []
for image in summaries:
    if ((image.key.startswith('image-asset/taken') and image.key.endswith('.JPG')) or (image.key.endswith('.jpg'))):
        images.append(image.key)
print(f'Total Number of images in our S3 bucket: {len(images)}')
print(f'List of image names:\n {images}')

Total Number of images in our S3 bucket: 104
List of image names:
 ['image-asset/1684177591-huge.jpg', 'image-asset/abdul-zreika-pMzlO7xJAd4-unsplash.jpg', 'image-asset/ahsanization-wpvEMgFV4w0-unsplash.jpg', 'image-asset/albany-capture-7o4t241DizQ-unsplash.jpg', 'image-asset/baptiste-pCH5-U4IBZU-unsplash.jpg', 'image-asset/bianca-sbircea-constantin-I-p1wEjKOTk-unsplash.jpg', 'image-asset/clem-onojeghuo-zZza888FSKg-unsplash.jpg', 'image-asset/dean-bennett-aBV8pVODWiM-unsplash.jpg', 'image-asset/emma-houghton-EixJzIdl4bc-unsplash.jpg', 'image-asset/emmanuel-ikwuegbu-KHO_jvns5Xc-unsplash.jpg', 'image-asset/emmanuel-ikwuegbu-zWOgsj3j0wA-unsplash.jpg', 'image-asset/eric-wang-umD2Bj4FmMU-unsplash.jpg', 'image-asset/gordianus-ernesto-ogTklVn1vpQ-unsplash.jpg', 'image-asset/irma-yanti-FC1RUP273r8-unsplash.jpg', 'image-asset/istrfry-marcus-pjW5WezrdJk-unsplash.jpg', 'image-asset/james-kovin-qqLxF3M-MA8-unsplash.jpg', 'image-asset/jeriden-villegas-VLPUm5wP5Z0-unsplash.jpg', 'image-asset/jeriden

Here is one example image from `images`.\
All of our images has public access.\
And this following image can be access with this link: [Picture of Jeremy M.](https://qtm350-fp.s3.amazonaws.com/image-asset/taken/IMG_8038.JPG).
<img src="https://qtm350-fp.s3.amazonaws.com/image-asset/taken/IMG_8038.JPG">

### Step 2. Test function with test image.

We will now test `.detect_protective_equipment()` from Rekognition with this example image above.

In [13]:
test = images[59]
print(f'https://qtm350-fp.s3.amazonaws.com/{images[59]}')

https://qtm350-fp.s3.amazonaws.com/image-asset/taken/IMG_8038.JPG


In [14]:
ppe_response = client.detect_protective_equipment(Image={'S3Object':{'Bucket':"qtm350-fp",'Name':f"{test}"}})

In [18]:
print(len(ppe_response['Persons']))

1


In [17]:
ppe_response

{'ProtectiveEquipmentModelVersion': '1.0',
 'Persons': [{'BodyParts': [{'Name': 'FACE',
     'Confidence': 98.63581085205078,
     'EquipmentDetections': [{'BoundingBox': {'Width': 0.04409034177660942,
        'Height': 0.05509009212255478,
        'Left': 0.4701075255870819,
        'Top': 0.5598670244216919},
       'Confidence': 99.9937515258789,
       'Type': 'FACE_COVER',
       'CoversBodyPart': {'Confidence': 72.3757095336914, 'Value': True}}]},
    {'Name': 'LEFT_HAND',
     'Confidence': 98.03620910644531,
     'EquipmentDetections': []},
    {'Name': 'RIGHT_HAND',
     'Confidence': 97.59645080566406,
     'EquipmentDetections': []},
    {'Name': 'HEAD',
     'Confidence': 99.91152954101562,
     'EquipmentDetections': []}],
   'BoundingBox': {'Width': 0.18333333730697632,
    'Height': 0.512499988079071,
    'Left': 0.4020833373069763,
    'Top': 0.48515623807907104},
   'Confidence': 99.94629669189453,
   'Id': 0}],
 'ResponseMetadata': {'RequestId': '13418f91-59e7-4cca-93

We can see from `ppe_response` that `.detect_protective_equipment()` returns the number of people inside the photo, as well as the labeling of the specific parts of the body where PPE can be worn and respective confidence scores.

### Step 3. Execute `detect_protective_equipment()` on every image, extract necessary data, and store it as .csv file.

We first import `pandas`, in which we will use `DataFrame` to store our data and finally export it to .csv using `to_csv()` function.

In [19]:
import pandas as pd

We create an empty dataframe with columns for values we want to store.

In [22]:
df = pd.DataFrame(columns=['img','bodyPartName', 'bodyPartConf', 'eq_type', 'eq_conf', 'eqCover_conf', 'eq_box', 'eqCover_bool'])

We run a for loop for each image in our `images` list, extracting and storing the fields we need to our dataframe `df`.\
In addition, we multiply `Width` and `Height` of `BoundingBox` to store the area of `BoundingBox` to `df['eq_box']`.

In [25]:
err_count = 0
for image in images:
    try:
        output = client.detect_protective_equipment(Image={'S3Object':{'Bucket':"qtm350-fp",'Name':f"{image}"}})
        persons = output['Persons']
        for person in persons:
            bodyParts= person['BodyParts']
            for bodypart in bodyParts:
                name = bodypart['Name']
                conf = bodypart['Confidence']
                equipmentDetection = bodypart['EquipmentDetections']
                if (len(equipmentDetection)==0):
                    continue
                else:
                    for eq in equipmentDetection:
                        eq_conf = eq['Confidence']
                        eq_type = eq['Type']
                        eqCover_conf = eq['CoversBodyPart']['Confidence']
                        eq_box = eq['BoundingBox']['Width']*eq['BoundingBox']['Height']
                        eqCover_bool = eq['CoversBodyPart']['Value']

                        df = df.append({
                                        'img':image,
                                        'bodyPartName':name,
                                        'bodyPartConf':conf,
                                        'eq_type':eq_type,
                                        'eq_conf':eq_conf,
                                        'eqCover_conf':eqCover_conf,
                                        'eq_box':eq_box,
                                        'eqCover_bool':eqCover_bool,
                                       }, ignore_index=True)
    except Exception: # exception case for image type errors
        err_count = err_count+1
        #print("error")
        pass

Here is the output `df` from the for loop above.

In [31]:
df

,img,bodyPartName,bodyPartConf,eq_type,eq_conf,eqCover_conf,eq_box,eqCover_bool
0,image-asset/1684177591-huge.jpg,FACE,99.458809,FACE_COVER,99.934128,90.723564,0.019554,True
1,image-asset/1684177591-huge.jpg,HEAD,99.969254,HEAD_COVER,68.631821,95.665405,0.039268,True
2,image-asset/1684177591-huge.jpg,FACE,96.018814,FACE_COVER,99.720436,89.325356,0.009923,True
3,image-asset/1684177591-huge.jpg,HEAD,99.608627,HEAD_COVER,95.798180,98.066414,0.051126,True
4,image-asset/1684177591-huge.jpg,FACE,94.266830,FACE_COVER,99.801407,96.197754,0.010404,True
...,...,...,...,...,...,...,...,...
120,image-asset/taken/IMG_8072.JPG,FACE,99.981300,FACE_COVER,95.747658,99.999992,0.012923,False
121,image-asset/taken/IMG_8073.JPG,HEAD,92.567757,HEAD_COVER,54.348499,100.000000,0.012487,False
122,image-asset/taken/IMG_8073.JPG,HEAD,98.892235,HEAD_COVER,61.682014,99.991692,0.001323,True
123,image-asset/taken/IMG_8074.JPG,FACE,99.362358,FACE_COVER,91.241142,53.143822,0.013208,True


Each detection of a `bodyPart` and its results are stored as one row of a dataframe.\
Now we will save this dataframe as `detect_ppe_output.csv`.

In [27]:
df.to_csv("detect_ppe_output.csv", index=False)